### This notebook shows examples of how to use the cyclops.query API on MIMIC-IV.

## Example 1. Get all patient encounters from 2015 (approximate year of care).

In [15]:
from sqlalchemy import select

from cyclops.query import mimic
from cyclops.query import utils as query_utils
from cyclops.query.interface import QueryInterface

patients = mimic.patients()
patients.run()
print(f"{len(patients.data)} rows extracted!")

2022-05-04 09:21:46,872 INFO cyclops.orm     - Query returned successfully!
2022-05-04 09:21:46,873 INFO cyclops.utils.profile - Finished executing function run_query in 6.861594 s


523740 rows extracted!


In [20]:
patients.data["subject_id"].nunique()

256878

## Example 2. Get all patient encounters with diagnoses (`schizophrenia` in ICD long title), in the year 2015.

In [2]:
patients = mimic.patients(years=[2015])
patients_schizophrenia = mimic.diagnoses(substring="schizophrenia", patients=patients)
patients_schizophrenia.run()
print(f"{len(patients_schizophrenia.data)} rows extracted!")
patients_schizophrenia.data["icd_title"].value_counts()

2022-05-04 01:25:46,805 INFO cyclops.orm     - Query returned successfully!
2022-05-04 01:25:46,806 INFO cyclops.utils.profile - Finished executing function run_query in 0.852207 s


802 rows extracted!


Schizophrenia, unspecified                                                 380
Unspecified schizophrenia, unspecified                                     179
Paranoid schizophrenia                                                      77
Paranoid type schizophrenia, unspecified                                    51
Catatonic schizophrenia                                                     21
Unspecified schizophrenia, chronic                                          17
Paranoid type schizophrenia, chronic with acute exacerbation                12
Paranoid type schizophrenia, chronic                                        12
Other schizophrenia                                                         10
Other specified types of schizophrenia, unspecified                          8
Catatonic type schizophrenia, unspecified                                    7
Unspecified schizophrenia, chronic with acute exacerbation                   4
Disorganized schizophrenia                          

## Example 3. Advanced - uses regexp_match from sqlalchemy. Get all patient encounters with diagnoses (ICD long title contains `schizophrenia` and `chronic` ), in the year 2015.

In [3]:
patients = mimic.patients(years=[2015])
diagnoses = mimic.diagnoses(patients=patients)
subquery = (
    select(diagnoses.query)
    .where(diagnoses.query.c.icd_title.regexp_match("(?=.*schizophrenia)(?=.*chronic)"))
    .subquery()
)
patients_schizophrenia_chronic = diagnoses
patients_schizophrenia_chronic.query = subquery
patients_schizophrenia_chronic.run()
print(f"{len(patients_schizophrenia_chronic.data)} rows extracted!")

2022-05-04 01:25:48,172 INFO cyclops.orm     - Query returned successfully!
2022-05-04 01:25:48,173 INFO cyclops.utils.profile - Finished executing function run_query in 1.347949 s


57 rows extracted!


## Example 4. Advanced - uses cyclops.query.utils.starts_with helper function. Get all patient encounters with diagnoses (starts with `Paranoid` in ICD long title), in the year 2015.

In [4]:
patients = mimic.patients(years=[2015])
diagnoses = mimic.diagnoses(patients=patients)
subquery = (
    select(diagnoses.query)
    .where(query_utils.starts_with(diagnoses.query.c.icd_title, "Paranoid"))
    .subquery()
)
patients_schizophrenia = diagnoses
patients_schizophrenia.query = subquery
patients_schizophrenia.run()
print(f"{len(patients_schizophrenia.data)} rows extracted!")

2022-05-04 01:25:48,994 INFO cyclops.orm     - Query returned successfully!
2022-05-04 01:25:48,995 INFO cyclops.utils.profile - Finished executing function run_query in 0.803267 s


170 rows extracted!


## Example 5. Get all patient encounters with diagnoses (ICD code is F209).

In [5]:
patients = mimic.patients()
patients_f209 = mimic.diagnoses(diagnosis_codes=["F209"], patients=patients)
patients_f209.run()
print(f"{len(patients_f209.data)} rows extracted!")

2022-05-04 01:25:50,880 INFO cyclops.orm     - Query returned successfully!
2022-05-04 01:25:50,881 INFO cyclops.utils.profile - Finished executing function run_query in 1.866347 s


1442 rows extracted!


## Example 6. Get all patient encounters with diagnoses (`delirium` in ICD long title).

In [6]:
patients = mimic.patients()
patients_delirium = mimic.diagnoses(substring="delirium", patients=patients)
patients_delirium.run()
print(f"{len(patients_delirium.data)} rows extracted!")

2022-05-04 01:25:51,806 INFO cyclops.orm     - Query returned successfully!
2022-05-04 01:25:51,807 INFO cyclops.utils.profile - Finished executing function run_query in 0.914249 s


9087 rows extracted!


## Example 7. Get routine vital signs for patients from year 2015, limit to 100 rows.

In [7]:
patients = mimic.patients(years=[2015])
patients_vitals = mimic.events(category="Routine Vital Signs", patients=patients)
patients_vitals.run(limit=100)
print(f"{len(patients_vitals.data)} rows extracted!")

2022-05-04 01:26:52,442 INFO cyclops.orm     - Query returned successfully!
2022-05-04 01:26:52,443 INFO cyclops.utils.profile - Finished executing function run_query in 60.614078 s


100 rows extracted!


## Example 8. Get hemoglobin lab tests for patients from year 2009, limit to 100 rows.

In [8]:
patients = mimic.patients(years=[2009])
patients_hemo_labs = mimic.events(
    category="labs", patients=patients, names="hemoglobin"
)
patients_hemo_labs.run(limit=100)
print(f"{len(patients_hemo_labs.data)} rows extracted!")

2022-05-04 01:27:19,571 INFO cyclops.orm     - Query returned successfully!
2022-05-04 01:27:19,574 INFO cyclops.utils.profile - Finished executing function run_query in 27.121478 s


100 rows extracted!


## Example 9. Get all lab events for patients from year 2010, that match substring "sodium", limit to 100 rows.

In [9]:
patients = mimic.patients(years=[2009])
patients_sodium_labs = mimic.events(
    category="labs", patients=patients, substring="sodium"
)
patients_sodium_labs.run(limit=100)
print(f"{len(patients_sodium_labs.data)} rows extracted!")

2022-05-04 01:27:48,997 INFO cyclops.orm     - Query returned successfully!
2022-05-04 01:27:49,000 INFO cyclops.utils.profile - Finished executing function run_query in 29.415685 s


100 rows extracted!


## Example 10. Get respiratory events for patients from year 2009, limit to 100 rows.

In [10]:
patients = mimic.patients(years=[2009])
patients_respiratory = mimic.events(category="Respiratory", patients=patients)
patients_respiratory.run(limit=100)
print(f"{len(patients_respiratory.data)} rows extracted!")

2022-05-04 01:28:54,911 INFO cyclops.orm     - Query returned successfully!
2022-05-04 01:28:54,915 INFO cyclops.utils.profile - Finished executing function run_query in 65.903820 s


100 rows extracted!


## Example 11. Get heart rate measurements of patients with delirium diagnoses, limit to 1000 rows.

In [11]:
patients = mimic.patients()
patients_delirium = mimic.diagnoses(substring="delirium", patients=patients)
patients_delirium_heart_rate = mimic.events(
    names="Heart Rate", patients=patients_delirium
)
patients_delirium_heart_rate.run(limit=1000)
print(f"{len(patients_delirium_heart_rate.data)} rows extracted!")

2022-05-04 01:29:24,509 INFO cyclops.orm     - Query returned successfully!
2022-05-04 01:29:24,514 INFO cyclops.utils.profile - Finished executing function run_query in 29.581928 s


1000 rows extracted!


## Example 12. Get d_items dimension table, for quick reference.

In [12]:
event_lookup_table = mimic.get_lookup_table("event_labels")
lookup_data = event_lookup_table.run()
print(f"{len(lookup_data)} rows extracted!")

2022-05-04 01:29:24,533 INFO cyclops.orm     - Query returned successfully!
2022-05-04 01:29:24,534 INFO cyclops.utils.profile - Finished executing function run_query in 0.015004 s


3861 rows extracted!
